In [20]:
import pandas as pd

In [21]:
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, ForeignKey, extract, text, event
from sqlalchemy.orm import relationship, declarative_base, sessionmaker


In [35]:
username1 = 'joab1'
username2 = 'joab2'
password = 'test'
host = 'localhost'
database = 'NBA'
databaseO = 'NBAORIGINAL'
port = '3307'

dim_engine = create_engine(f"mysql+pymysql://{username1}:{password}@{host}/{database}")
trans_engine = create_engine(f"mysql+pymysql://{username2}:{password}@{host}:{port}/{databaseO}")

TransSession = sessionmaker(bind=trans_engine)
DimSession = sessionmaker(bind=dim_engine)

trans_session = TransSession()
dim_session = DimSession()



In [36]:
Base = declarative_base()

# Dimension Tables

class DimPlayerInfo(Base):
    __tablename__ = 'dim_player_info'
    
    PLAYER_ID = Column(Integer, primary_key=True)
    PLAYER_NAME = Column(String(100))

class DimTeam(Base):
    __tablename__ = 'dim_team'
    
    TEAM_ID = Column(Integer, primary_key=True)
    LEAGUE_ID = Column(Integer)
    MIN_YEAR = Column(Integer)
    MAX_YEAR = Column(Integer)
    ABBREVIATION = Column(String(100))
    NICKNAME = Column(String(100))
    YEARFOUNDED = Column(Integer)
    CITY = Column(String(100))
    ARENA = Column(String(100))
    ARENACAPACITY = Column(Float)
    OWNER = Column(String(100))
    GENERALMANAGER = Column(String(100))
    HEADCOACH = Column(String(100))
    DLEAGUEAFFILIATION = Column(String(100))

class DimSeason(Base):
    __tablename__ = 'dim_season'
    
    SEASON = Column(Integer, primary_key=True)

class DimPlayer(Base):
    __tablename__ = 'dim_player'
    
    ASSOCIATION_ID = Column(Integer, primary_key=True, autoincrement=True)
    PLAYER_ID = Column(Integer, ForeignKey('dim_player_info.PLAYER_ID'))
    TEAM_ID = Column(Integer, ForeignKey('dim_team.TEAM_ID'))
    SEASON = Column(Integer, ForeignKey('dim_season.SEASON'))




class DimGame(Base):
    __tablename__ = 'dim_game'
    
    GAME_ID = Column(Integer, primary_key=True)
    GAME_DATE_EST = Column(Date)
    YEAR = Column(Integer)
    MONTH_NAME = Column(String(20))
    DAY_NAME = Column(String(20))
    SEASON = Column(Integer, ForeignKey('dim_season.SEASON'))
    GAME_STATUS_TEXT = Column(String(100))
    HOME_TEAM_ID = Column(Integer, ForeignKey('dim_team.TEAM_ID'))
    VISITOR_TEAM_ID = Column(Integer, ForeignKey('dim_team.TEAM_ID'))




# Fact Tables
class FactPlayerMetrics(Base):
    __tablename__ = 'fact_player_metrics'
    
    METRIC_ID = Column(Integer, primary_key=True, autoincrement=True)
    GAME_ID = Column(Integer, ForeignKey('dim_game.GAME_ID'))
    PLAYER_ID = Column(Integer, ForeignKey('dim_player.PLAYER_ID'))
    PLAYER_PTS = Column(Float)
    PLAYER_FGM = Column(Float)
    PLAYER_FGA = Column(Float)
    PLAYER_FG_PCT = Column(Float)
    PLAYER_FG3M = Column(Float)
    PLAYER_FG3A = Column(Float)
    PLAYER_FG3_PCT = Column(Float)
    PLAYER_FTM = Column(Float)
    PLAYER_FTA = Column(Float)
    PLAYER_FT_PCT = Column(Float)
    PLAYER_OREB = Column(Float)
    PLAYER_DREB = Column(Float)
    PLAYER_REB = Column(Float)
    PLAYER_AST = Column(Float)
    PLAYER_STL = Column(Float)
    PLAYER_BLK = Column(Float)
    PLAYER_TO = Column(Float)
    PLAYER_PF = Column(Float)
    PLAYER_PLUS_MINUS = Column(Float)

class FactGameMetrics(Base):
    __tablename__ = 'fact_game_metrics'
    
    FACT_ID = Column(Integer, primary_key=True, autoincrement=True)
    GAME_ID = Column(Integer, ForeignKey('dim_game.GAME_ID'))
    HOME_TEAM_WINS = Column(Integer)

# Create an engine and bind it
Base.metadata.create_all(dim_engine)



In [37]:
@event.listens_for(DimGame, 'before_insert')
@event.listens_for(DimGame, 'before_update')
def extract_date_components(mapper, connection, target):
    if target.GAME_DATE_EST:
        date_obj = target.GAME_DATE_EST  # esto debería ser un objeto date
        target.YEAR = date_obj.year
        target.MONTH = date_obj.month
        target.MONTH_NAME = date_obj.strftime('%B')
        target.DAY = date_obj.day
        target.DAY_NAME = date_obj.strftime('%A')


In [38]:
#1.- dim_player_info

#Del modelo transaccional al DM
# SQL para consultar jugadores únicos de la base de datos transaccional
sql_query = text("""
SELECT PLAYER_ID, MAX(PLAYER_NAME) AS PLAYER_NAME
FROM players
GROUP BY PLAYER_ID
""")

# Obtén una conexión de la base de datos transaccional y ejecuta la consulta
with trans_engine.connect() as connection:
    result = connection.execute(sql_query)

    # Extrae los datos de la consulta
    unique_players = result.fetchall()

    # Ahora inserta esos datos en la DM
    insert_sql = text("""
    INSERT INTO dim_player_info (PLAYER_ID, PLAYER_NAME)
    VALUES (:player_id, :player_name)
    """)

    # Usando una conexión a la base de datos dimensional, inserta los datos
    with dim_engine.connect() as dim_conn:
        for player_id, player_name in unique_players:
            dim_conn.execute(insert_sql, {"player_id": player_id, "player_name": player_name})
        dim_conn.commit()  # Confirma los cambios en la base de datos


In [39]:
#2.- dim_team

# SQL para consultar equipos únicos de la base de datos transaccional
sql_query_teams = text("""
SELECT DISTINCT 
    TEAM_ID, LEAGUE_ID, MIN_YEAR, MAX_YEAR, ABBREVIATION, NICKNAME, YEARFOUNDED, CITY,
    ARENA, ARENACAPACITY, OWNER, GENERALMANAGER, HEADCOACH, DLEAGUEAFFILIATION
FROM teams
""")

# Obtén una conexión de la base de datos transaccional y ejecuta la consulta
with trans_engine.connect() as connection:
    result = connection.execute(sql_query_teams)

    # Extrae los datos de la consulta
    unique_teams = result.fetchall()

    # Ahora inserta esos datos en la DM
    insert_sql_teams = text("""
    INSERT INTO dim_team (
        TEAM_ID, LEAGUE_ID, MIN_YEAR, MAX_YEAR, ABBREVIATION, NICKNAME, YEARFOUNDED, CITY,
        ARENA, ARENACAPACITY, OWNER, GENERALMANAGER, HEADCOACH, DLEAGUEAFFILIATION
    )
    VALUES (
        :team_id, :league_id, :min_year, :max_year, :abbreviation, :nickname, :yearfounded, :city,
        :arena, :arenacapacity, :owner, :generalmanager, :headcoach, :dleagueaffiliation
    )
    """)

    # Usando una conexión a la base de datos dimensional, inserta los datos
    with dim_engine.connect() as dim_conn:
        for team in unique_teams:
            dim_conn.execute(
                insert_sql_teams,
                {
                    'team_id': team[0],
                    'league_id': team[1],
                    'min_year': team[2],
                    'max_year': team[3],
                    'abbreviation': team[4],
                    'nickname': team[5],
                    'yearfounded': team[6],
                    'city': team[7],
                    'arena': team[8],
                    'arenacapacity': team[9],
                    'owner': team[10],
                    'generalmanager': team[11],
                    'headcoach': team[12],
                    'dleagueaffiliation': team[13]
                }
            )
        dim_conn.commit()

In [40]:
#3.- dim_season

# 1. SQL para consultar temporadas únicas de la base de datos transaccional
sql_query_seasons = text("""
SELECT DISTINCT SEASON
FROM games
""")

# Obtén una conexión de la base de datos transaccional y ejecuta la consulta
with trans_engine.connect() as connection:
    result = connection.execute(sql_query_seasons)

    # Extrae los datos de la consulta
    unique_seasons = result.fetchall()

    # 2. Ahora inserta esos datos en la DM
    insert_sql_seasons = text("""
    INSERT INTO dim_season (SEASON)
    VALUES (:season)
    """)

    # Usando una conexión a la base de datos dimensional, inserta los datos
    with dim_engine.connect() as dim_conn:
        for season, in unique_seasons:  # Nota: la coma después de 'season' es para desempaquetar el tuple
            dim_conn.execute(
                insert_sql_seasons,
                {'season': season}
            )
        dim_conn.commit()

In [41]:
#4.- dim_player
# 1. SQL para consultar jugadores únicos de players
sql_query_players = text("""
SELECT DISTINCT PLAYER_ID, TEAM_ID, SEASON
FROM players
""")

# 2. SQL para consultar jugadores únicos de game_details (sin 'SEASON')
sql_query_details = text("""
SELECT DISTINCT PLAYER_ID, TEAM_ID
FROM game_details
""")

# Extrae los datos de las tablas de la base de datos transaccional
with trans_engine.connect() as connection:
    players_df = pd.read_sql(sql_query_players, connection)
    details_df = pd.read_sql(sql_query_details, connection)

# Combina y filtra los registros usando Pandas
# Merge players_df and details_df on both PLAYER_ID and TEAM_ID using a left join
merged_df = pd.merge(details_df, players_df, on=['PLAYER_ID', 'TEAM_ID'], how='left')

# Elimina duplicados basados en PLAYER_ID, TEAM_ID y SEASON
final_df = merged_df.drop_duplicates(subset=['PLAYER_ID', 'TEAM_ID', 'SEASON'])

# Inserta los registros combinados y filtrados en la tabla dimensional
final_df.to_sql('dim_player', dim_engine, if_exists='append', index=False)


26445

In [42]:
#5.- dim_game
# SQL para seleccionar datos únicos de la tabla games
sql_query_games = text("""
SELECT DISTINCT
    GAME_ID, GAME_DATE_EST, SEASON, GAME_STATUS_TEXT, HOME_TEAM_ID, VISITOR_TEAM_ID
FROM games
""")

# Obtén una conexión de la base de datos transaccional y ejecuta la consulta
with trans_engine.connect() as connection:
    result = connection.execute(sql_query_games)
    unique_games = result.fetchall()

# SQL para insertar en dim_game
insert_sql_games = text("""
INSERT INTO dim_game
    (GAME_ID, GAME_DATE_EST, SEASON, GAME_STATUS_TEXT, HOME_TEAM_ID, VISITOR_TEAM_ID)
VALUES
    (:game_id, :game_date_est, :season, :game_status_text, :home_team_id, :visitor_team_id)
""")

# Usando una conexión a la base de datos dimensional, inserta los datos
with dim_engine.connect() as dim_conn:
    for game in unique_games:
        dim_conn.execute(
            insert_sql_games,
            {
                'game_id': game[0],
                'game_date_est': game[1],
                'season': game[2],
                'game_status_text': game[3],
                'home_team_id': game[4],
                'visitor_team_id': game[5]
            }
        )
    dim_conn.commit()

In [43]:
#5.- rellenar las columnas year, month, day
# Obtener todos los registros
games = dim_session.query(DimGame).all()

for game in games:
    if game.GAME_DATE_EST:
        game.YEAR = game.GAME_DATE_EST.year
        game.MONTH = game.GAME_DATE_EST.month
        game.DAY = game.GAME_DATE_EST.day

# Hacer commit de los cambios
dim_session.commit()


In [44]:
#6.- fact_player_metrics

# 1. SQL para transformar y consultar las métricas del jugador
sql_query_player_metrics = text("""
SELECT 
    GAME_ID AS GAME_ID,
    PLAYER_ID AS PLAYER_ID,
    PTS AS PLAYER_PTS,
    FGM AS PLAYER_FGM,
    FGA AS PLAYER_FGA,
    FG_PCT AS PLAYER_FG_PCT,
    FG3M AS PLAYER_FG3M,
    FG3A AS PLAYER_FG3A,
    FG3_PCT AS PLAYER_FG3_PCT,
    FTM AS PLAYER_FTM,
    FTA AS PLAYER_FTA,
    FT_PCT AS PLAYER_FT_PCT,
    OREB AS PLAYER_OREB,
    DREB AS PLAYER_DREB,
    REB AS PLAYER_REB,
    AST AS PLAYER_AST,
    STL AS PLAYER_STL,
    BLK AS PLAYER_BLK,
    `TO` AS PLAYER_TO,
    PF AS PLAYER_PF,
    PLUS_MINUS AS PLAYER_PLUS_MINUS
FROM game_details
""")

# 2. Conectar a la base de datos transaccional y ejecutar la consulta
with trans_engine.connect() as connection:
    player_metrics_result = connection.execute(sql_query_player_metrics)
    player_metrics_data = player_metrics_result.fetchall()

# 3. SQL para insertar datos en la tabla player_metrics del DM
insert_sql_player_metrics = text("""
INSERT INTO fact_player_metrics 
(GAME_ID, PLAYER_ID, PLAYER_PTS, PLAYER_FGM, PLAYER_FGA, PLAYER_FG_PCT, PLAYER_FG3M, PLAYER_FG3A, PLAYER_FG3_PCT,
 PLAYER_FTM, PLAYER_FTA, PLAYER_FT_PCT, PLAYER_OREB, PLAYER_DREB, PLAYER_REB, PLAYER_AST, PLAYER_STL, PLAYER_BLK, 
 PLAYER_TO, PLAYER_PF, PLAYER_PLUS_MINUS)
VALUES 
(:game_id, :player_id, :player_pts, :player_fgm, :player_fga, :player_fg_pct, :player_fg3m, :player_fg3a, :player_fg3_pct,
 :player_ftm, :player_fta, :player_ft_pct, :player_oreb, :player_dreb, :player_reb, :player_ast, :player_stl, :player_blk, 
 :player_to, :player_pf, :player_plus_minus)
""")

# 4. Conectar a la base de datos dimensional e insertar los datos
with dim_engine.connect() as dim_conn:
    for data in player_metrics_data:
        dim_conn.execute(
            insert_sql_player_metrics,
            {
                'game_id': data[0],
                'player_id': data[1],
                'player_pts': data[2],
                'player_fgm': data[3],
                'player_fga': data[4],
                'player_fg_pct': data[5],
                'player_fg3m': data[6],
                'player_fg3a': data[7],
                'player_fg3_pct': data[8],
                'player_ftm': data[9],
                'player_fta': data[10],
                'player_ft_pct': data[11],
                'player_oreb': data[12],
                'player_dreb': data[13],
                'player_reb': data[14],
                'player_ast': data[15],
                'player_stl': data[16],
                'player_blk': data[17],
                'player_to': data[18],
                'player_pf': data[19],
                'player_plus_minus': data[20]
            }
        )
    dim_conn.commit()

In [45]:
# 7.- fact_game_metrics

# 1. Consulta SQL para seleccionar datos de la base de datos transaccional
sql_query_games = text("""
    SELECT
        GAME_ID,
        HOME_TEAM_WINS
    FROM games
""")

# Obtén una conexión de la base de datos transaccional y ejecuta la consulta
with trans_engine.connect() as connection:
    result = connection.execute(sql_query_games)
    game_data = result.fetchall()

# 2. SQL para insertar en fact_game_metrics
insert_sql = text("""
    INSERT INTO fact_game_metrics
    (GAME_ID, HOME_TEAM_WINS)
    VALUES
    (:game_id, :home_team_wins)
""")

with dim_engine.connect() as dim_conn:
    for game in game_data:
        dim_conn.execute(
            insert_sql,
            {
                'game_id': game[0],
                'home_team_wins': game[1]
            }
        )
    dim_conn.commit()

In [9]:
dim_engine.dispose()
trans_engine.dispose()

In [34]:
#No ejecutar a menos que valores tu vida
#Base.metadata.drop_all(dim_engine)